In [1]:
!pip install mwparserfromhell

In [ ]:
# Run block in tmux
import os
from pathlib import Path
from datasets import load_dataset, DownloadConfig
import numpy as np
from transformers import AutoTokenizer
from chunk import chunk_and_tokenize

output_dir = Path('/') / 'mnt' / 'ssd-1' / 'lucia'

# dataset = load_dataset(
#     "allenai/c4", 
#     "es", 
#     split='train', 
#     download_config=DownloadConfig(resume_download=True),
#     cache_dir=f"/mnt/ssd-1/hf_cache")

dataset = load_dataset(
    "spanish_billion_words", 
    split='train', 
    download_config=DownloadConfig(resume_download=True),
    cache_dir=Path('/') / 'mnt' / 'ssd-1' / 'hf_cache')

model_name = "pythia-160m"
tokenizer = AutoTokenizer.from_pretrained(f"EleutherAI/{model_name}")   
# filtered_dataset = dataset.shuffle().select(range(int(3.5e11)))
shuffled = dataset.shuffle()
data, bpb_ratio = chunk_and_tokenize(shuffled, tokenizer, max_length=2049)

data.save_to_disk(output_dir / 'es_1b_full_tokenized.hf')

fp = np.memmap(output_dir / 'es_1b_full.bin', dtype=np.uint16, mode='w+', shape=(len(data), 2049))
for i, item in enumerate(data):
    fp[i] = item['input_ids'].numpy()   
fp.flush()


In [ ]:
# es = load_dataset("allenai/c4", "es", split='train')
es = load_dataset("es/pile-10k", split='train')

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")

data, bpb_ratio = chunk_and_tokenize(es, tokenizer, max_length=2049)
data.save_to_disk("es_tokenized.hf")

fp = np.memmap('es.bin', dtype=np.uint16, mode='w+', shape=(len(data), 2049))
for i, item in enumerate(data):
    fp[i] = item['input_ids'].numpy()
fp.flush()

print(len(data))
print(data["length"][0:6])
print(len(data["input_ids"][0]))
print(sum(data["length"]))
print(os.getcwd())

In [ ]:
# first = 0
# sec = 0
# third = 0
# four = 0
# for item in data['length']:
#     if item == 2048:
#         first += 1
#     elif item == 2049:
#         sec += 1
#     elif item == 2050:
#         third += 1
#     elif item == 2051:
#         four += 1
#     else:
#         print(item)

# print(first, sec, third, four)